<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/CODIGO_ANONIMIZACION_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Librerías necesarias
!pip install transformers
!pip install faker
!pip install gender-guesser

import pandas as pd
from transformers import pipeline
from faker import Faker
import gender_guesser.detector as gender
import unicodedata
import string
import re

# Inicializar Faker para español y el detector de género
fake = Faker('es_ES')
gender_detector = gender.Detector()

# Cargar el modelo de Hugging Face para NER en español con estrategia de agregación (BERT)
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")

# Función para eliminar acentos y caracteres especiales
def eliminar_acentos_y_puntuacion(texto):
    texto_sin_acentos = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    return texto_sin_acentos.translate(str.maketrans('', '', string.punctuation))

# Función para generar un nombre falso manteniendo el género para personas (PER)
def get_gendered_fake_name(original_name):
    first_name = original_name.split()[0]
    detected_gender = gender_detector.get_gender(first_name)
    if detected_gender in ['male', 'mostly_male']:
        return fake.first_name_male()
    elif detected_gender in ['female', 'mostly_female']:
        return fake.first_name_female()
    else:
        return fake.first_name()

# Función para anonimizar números de teléfono
def anonimizar_telefono(texto):
    # Busca patrones comunes de números de teléfono
    return re.sub(r'\b\d{9}\b', lambda x: fake.phone_number(), texto)

# Función para anonimizar DNI
def anonimizar_dni(texto):
    # Busca patrones comunes de DNI en España (8 dígitos seguidos de una letra)
    return re.sub(r'\b\d{8}[A-Za-z]\b', lambda x: fake.ssn(), texto)

# Función principal para anonimizar el texto usando BERT y mantener consistencia en los nombres
def anonimizar_texto(texto, entity_replacements):
    texto_limpio = eliminar_acentos_y_puntuacion(texto)

    # Anonimizar números de teléfono y DNI antes de procesar con NER
    texto_limpio = anonimizar_telefono(texto_limpio)
    texto_limpio = anonimizar_dni(texto_limpio)

    # Identificar entidades con NER
    entidades = ner_model(texto_limpio)

    for ent in entidades:
        label = ent['entity_group']
        original_text = ent['word']

        # Anonimización de personas
        if label == "PER":
            if original_text not in entity_replacements:
                fake_name = get_gendered_fake_name(original_text)
                entity_replacements[original_text] = fake_name
            texto_limpio = texto_limpio.replace(original_text, entity_replacements[original_text])

    return texto_limpio

# Cargar el archivo CSV que contiene los chats
df = pd.read_csv('/content/chat_s1.csv', encoding='latin1')

# Diccionario para almacenar los reemplazos de entidades
entity_replacements = {}

# Aplicar la anonimización a cada fila del archivo
df['Texto_anonimizado'] = df['Texto'].apply(lambda x: anonimizar_texto(x, entity_replacements))

# Guardar el archivo completo con texto anonimizado
file_path_anonimizado = "/content/chats_psicologicos_anonimizados.csv"
df.to_csv(file_path_anonimizado, index=False)

# Crear un nuevo archivo que solo incluya los textos anonimizados
file_path_solo_anonimizado = "/content/texto_anonimizado_unico.csv"
df[['Participante','Texto_anonimizado']].to_csv(file_path_solo_anonimizado, index=False)

file_path_solo_anonimizado


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'/content/texto_anonimizado_unico.csv'